In [6]:
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players
import pandas as pd


In [2]:
# search for a player
lebron = players.find_players_by_full_name("LeBron James")[0]
lebron_id = lebron["id"]

print(lebron_id)


2544


In [3]:
shots = shotchartdetail.ShotChartDetail(
    team_id=0,                    # 0 = all teams
    player_id=lebron_id,
    season_type_all_star="Regular Season",
    season_nullable="2023-24",
    context_measure_simple="FGA"  # field goal attempts
)


In [4]:
df = shots.get_data_frames()[0]
df.head()


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022300015,7,2544,LeBron James,1610612747,Los Angeles Lakers,1,11,40,...,Center(C),16-24 ft.,17,-47,172,1,0,20231110,PHX,LAL
1,Shot Chart Detail,0022300015,11,2544,LeBron James,1610612747,Los Angeles Lakers,1,11,12,...,Center(C),Less Than 8 ft.,0,2,5,1,1,20231110,PHX,LAL
2,Shot Chart Detail,0022300015,20,2544,LeBron James,1610612747,Los Angeles Lakers,1,10,1,...,Left Side(L),8-16 ft.,12,-119,32,1,0,20231110,PHX,LAL
3,Shot Chart Detail,0022300015,36,2544,LeBron James,1610612747,Los Angeles Lakers,1,9,0,...,Center(C),Less Than 8 ft.,0,1,8,1,1,20231110,PHX,LAL
4,Shot Chart Detail,0022300015,72,2544,LeBron James,1610612747,Los Angeles Lakers,1,5,18,...,Left Side(L),8-16 ft.,12,-82,89,1,0,20231110,PHX,LAL


In [5]:
# % of shots that are 3s
three_rate = (df["SHOT_TYPE"] == "3PT Field Goal").mean()

# FG% by distance
fg_by_dist = df.groupby("SHOT_DISTANCE")["SHOT_MADE_FLAG"].mean()

print("3PT Rate:", three_rate)


3PT Rate: 0.2860520094562648


In [2]:
import time
import pandas as pd

from nba_api.stats.static import players
from nba_api.stats.endpoints import leaguegamefinder, shotchartdetail

# --- 1) season helper: last 10 completed Finals seasons (2015-16 .. 2024-25) ---
def last_10_completed_finals_seasons():
    # 2024-25 is the last completed season before Jan 2026
    end_start_year = 2025
    start_start_year = end_start_year - 9  # 10 seasons total
    return [f"{y}-{str(y+1)[-2:]}" for y in range(start_start_year, end_start_year + 1)]

# --- 2) robust request wrapper (rate limit + retry) ---
def safe_call(fn, tries=4, base_sleep=1.0):
    for i in range(tries):
        try:
            return fn()
        except Exception as e:
            if i == tries - 1:
                raise
            time.sleep(base_sleep * (2 ** i))

# --- 3) find player_id from name ---
def get_player_id(full_name: str) -> int:
    matches = players.find_players_by_full_name(full_name)
    if not matches:
        raise ValueError(f"No player found for name: {full_name}")
    # best match is usually first
    return matches[0]["id"]

# --- 4) Finals GAME_IDs the player appeared in (per season) ---
def get_player_finals_game_ids(player_id: int, season: str, sleep_s=0.8):
    # LeagueGameFinder supports player_id_nullable, season_type_nullable, and po_round_nullable. :contentReference[oaicite:1]{index=1}
    def _call():
        return leaguegamefinder.LeagueGameFinder(
            player_id_nullable=player_id,
            season_nullable=season,
            season_type_nullable="Playoffs",
            po_round_nullable=4,  # 4 = Finals round
        ).get_data_frames()[0]

    games = safe_call(_call)
    time.sleep(sleep_s)

    # If player made Finals that year, this will have 4–7 rows (one per game)
    if games.empty:
        return []

    # Extra safety: only keep actual playoff game ids (typically start with '004' for playoffs)
    # (Not strictly necessary since we asked Playoffs + PO round.)
    game_ids = sorted(games["GAME_ID"].unique().tolist())
    return game_ids

# --- 5) All Finals shot attempts for the player (per game) ---
def get_player_shots_for_game(player_id: int, season: str, game_id: str, sleep_s=0.8):
    # ShotChartDetail supports player_id, season_type_all_star, season_nullable, game_id_nullable, context_measure_simple. :contentReference[oaicite:2]{index=2}
    def _call():
        return shotchartdetail.ShotChartDetail(
            team_id=0,
            player_id=player_id,
            season_nullable=season,
            season_type_all_star="Playoffs",
            game_id_nullable=game_id,
            context_measure_simple="FGA"
        ).get_data_frames()[0]

    df = safe_call(_call)
    time.sleep(sleep_s)

    if not df.empty:
        df["SEASON"] = season
        df["FINALS_GAME_ID"] = game_id
    return df

# --- 6) full pipeline: ANY player -> last 10 seasons -> Finals only -> all FGAs ---
def finals_shots_last_10_seasons(player_full_name: str):
    player_id = get_player_id(player_full_name)
    seasons = last_10_completed_finals_seasons()

    out = []
    finals_games_index = []  # optional: store which seasons they reached Finals

    for season in seasons:
        game_ids = get_player_finals_game_ids(player_id, season)
        if game_ids:
            finals_games_index.append((season, len(game_ids)))

        for gid in game_ids:
            df_game = get_player_shots_for_game(player_id, season, gid)
            if not df_game.empty:
                out.append(df_game)

    shots_df = pd.concat(out, ignore_index=True) if out else pd.DataFrame()
    finals_summary = pd.DataFrame(finals_games_index, columns=["SEASON", "FINALS_GAMES_FOUND"])
    return shots_df, finals_summary

# ------------------ USAGE ------------------
shots_df, finals_summary = finals_shots_last_10_seasons("LeBron James")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2016-17                  18
1  2017-18                  22
2  2019-20                  21
3  2020-21                   6
4  2022-23                  16
5  2023-24                   5
6  2024-25                   5
Total Finals shot attempts rows: 1872


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041600111,97,2544,LeBron James,1610612739,Cleveland Cavaliers,1,2,33,...,0,-7,-1,1,0,20170415,CLE,IND,2016-17,0041600111
1,Shot Chart Detail,0041600111,113,2544,LeBron James,1610612739,Cleveland Cavaliers,1,1,40,...,0,-7,-6,1,1,20170415,CLE,IND,2016-17,0041600111
2,Shot Chart Detail,0041600111,141,2544,LeBron James,1610612739,Cleveland Cavaliers,2,10,48,...,0,-2,-1,1,1,20170415,CLE,IND,2016-17,0041600111
3,Shot Chart Detail,0041600111,159,2544,LeBron James,1610612739,Cleveland Cavaliers,2,9,14,...,0,0,3,1,1,20170415,CLE,IND,2016-17,0041600111
4,Shot Chart Detail,0041600111,190,2544,LeBron James,1610612739,Cleveland Cavaliers,2,6,30,...,0,0,1,1,1,20170415,CLE,IND,2016-17,0041600111
5,Shot Chart Detail,0041600111,280,2544,LeBron James,1610612739,Cleveland Cavaliers,2,0,1,...,0,-7,-1,1,1,20170415,CLE,IND,2016-17,0041600111
6,Shot Chart Detail,0041600111,290,2544,LeBron James,1610612739,Cleveland Cavaliers,3,11,16,...,7,-50,61,1,1,20170415,CLE,IND,2016-17,0041600111
7,Shot Chart Detail,0041600111,296,2544,LeBron James,1610612739,Cleveland Cavaliers,3,10,50,...,23,-235,-6,1,1,20170415,CLE,IND,2016-17,0041600111
8,Shot Chart Detail,0041600111,318,2544,LeBron James,1610612739,Cleveland Cavaliers,3,7,3,...,25,-173,190,1,1,20170415,CLE,IND,2016-17,0041600111
9,Shot Chart Detail,0041600111,346,2544,LeBron James,1610612739,Cleveland Cavaliers,3,5,5,...,11,22,115,1,0,20170415,CLE,IND,2016-17,0041600111


In [3]:
shots_df, finals_summary = finals_shots_last_10_seasons("Devin Booker")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2020-21                  22
1  2021-22                  10
2  2022-23                  11
3  2023-24                   4
Total Finals shot attempts rows: 955


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0042000151,9,1626164,Devin Booker,1610612756,Phoenix Suns,1,11,28,...,28,-202,196,1,0,20210523,PHX,LAL,2020-21,0042000151
1,Shot Chart Detail,0042000151,13,1626164,Devin Booker,1610612756,Phoenix Suns,1,10,59,...,1,-11,1,1,0,20210523,PHX,LAL,2020-21,0042000151
2,Shot Chart Detail,0042000151,17,1626164,Devin Booker,1610612756,Phoenix Suns,1,10,16,...,23,230,-13,1,1,20210523,PHX,LAL,2020-21,0042000151
3,Shot Chart Detail,0042000151,71,1626164,Devin Booker,1610612756,Phoenix Suns,1,5,23,...,0,-2,-3,1,1,20210523,PHX,LAL,2020-21,0042000151
4,Shot Chart Detail,0042000151,90,1626164,Devin Booker,1610612756,Phoenix Suns,1,3,46,...,0,8,-3,1,1,20210523,PHX,LAL,2020-21,0042000151
5,Shot Chart Detail,0042000151,127,1626164,Devin Booker,1610612756,Phoenix Suns,1,0,42,...,7,74,16,1,1,20210523,PHX,LAL,2020-21,0042000151
6,Shot Chart Detail,0042000151,130,1626164,Devin Booker,1610612756,Phoenix Suns,1,0,33,...,30,55,302,1,0,20210523,PHX,LAL,2020-21,0042000151
7,Shot Chart Detail,0042000151,209,1626164,Devin Booker,1610612756,Phoenix Suns,2,7,24,...,20,79,190,1,1,20210523,PHX,LAL,2020-21,0042000151
8,Shot Chart Detail,0042000151,220,1626164,Devin Booker,1610612756,Phoenix Suns,2,6,52,...,16,-160,34,1,0,20210523,PHX,LAL,2020-21,0042000151
9,Shot Chart Detail,0042000151,226,1626164,Devin Booker,1610612756,Phoenix Suns,2,6,11,...,6,-14,63,1,0,20210523,PHX,LAL,2020-21,0042000151


In [4]:
shots_df, finals_summary = finals_shots_last_10_seasons("Grant Williams")
print(finals_summary)
print("Total Finals shot attempts rows:", len(shots_df))
shots_df.head(50)


    SEASON  FINALS_GAMES_FOUND
0  2019-20                  17
1  2020-21                   5
2  2021-22                  24
3  2022-23                  15
Total Finals shot attempts rows: 241


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,SEASON,FINALS_GAME_ID
0,Shot Chart Detail,0041900122,84,1629684,Grant Williams,1610612738,Boston Celtics,1,4,36,...,26,-86,252,1,1,20200819,BOS,PHI,2019-20,0041900122
1,Shot Chart Detail,0041900122,388,1629684,Grant Williams,1610612738,Boston Celtics,3,4,37,...,26,-153,221,1,1,20200819,BOS,PHI,2019-20,0041900122
2,Shot Chart Detail,0041900123,128,1629684,Grant Williams,1610612738,Boston Celtics,1,2,56,...,5,43,26,1,0,20200821,PHI,BOS,2019-20,0041900123
3,Shot Chart Detail,0041900123,471,1629684,Grant Williams,1610612738,Boston Celtics,3,4,23,...,2,8,25,1,1,20200821,PHI,BOS,2019-20,0041900123
4,Shot Chart Detail,0041900123,483,1629684,Grant Williams,1610612738,Boston Celtics,3,3,14,...,24,1,248,1,1,20200821,PHI,BOS,2019-20,0041900123
5,Shot Chart Detail,0041900124,452,1629684,Grant Williams,1610612738,Boston Celtics,3,3,5,...,23,-234,0,1,1,20200823,PHI,BOS,2019-20,0041900124
6,Shot Chart Detail,0041900212,278,1629684,Grant Williams,1610612738,Boston Celtics,2,0,45,...,1,8,7,1,1,20200901,TOR,BOS,2019-20,0041900212
7,Shot Chart Detail,0041900212,399,1629684,Grant Williams,1610612738,Boston Celtics,3,4,49,...,2,22,19,1,0,20200901,TOR,BOS,2019-20,0041900212
8,Shot Chart Detail,0041900213,159,1629684,Grant Williams,1610612738,Boston Celtics,1,1,52,...,22,226,-5,1,0,20200903,BOS,TOR,2019-20,0041900213
9,Shot Chart Detail,0041900213,440,1629684,Grant Williams,1610612738,Boston Celtics,3,2,31,...,11,100,63,1,0,20200903,BOS,TOR,2019-20,0041900213


In [7]:
name = "Stephen Curry"

player = players.find_players_by_full_name(name)[0]
player_id = player['id']

df = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_data_frames()[0]

print(df[['DISPLAY_FIRST_LAST', 'HEIGHT', 'WEIGHT']].iloc[0])

DISPLAY_FIRST_LAST    Stephen Curry
HEIGHT                          6-2
WEIGHT                          185
Name: 0, dtype: str
